https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html

docker run -d -p 8888:8888 -e GRANT_SUDO=yes -v /home/mrthink/workspace/tensorflow:/data psygrammer/tensorflow_basic
: /data로 외부 data를 전달 할 수 있게 함.

# The MNIST Data


In [21]:
import os
result = os.popen("cp /data/mnist . -ar")

for line in result.readlines():
    print(line)

In [23]:
from mnist import *
mnist = input_data.read_data_sets("/data/mnist/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz


The downloaded data is split into two parts, 60,000 data points of training data (mnist.train) and 10,000 points of test data (mnist.test).
the training images are mnist.train.images and the train labels are mnist.train.labels

https://www.tensorflow.org/versions/master/images/MNIST-Matrix.png

Each image is 28 pixels by 28 pixels.
We can flatten this array into a vector of 28x28 = 784 numbers.

very rich structure???



In [30]:
# import urllib2
# import matplotlib.pyplot as plt

# f=urllib2.urlopen("https://www.tensorflow.org/versions/master/images/MNIST-Matrix.png")

# a = plt.imread(f)
# plt.imshow(a)
# plt.show()

# 이미지 어떻게 출력하지...??

Flattening the data throws away information about the 2D structure of the image. Isn't that bad? Well, the best computer vision methods do exploit this structure, and we will in later tutorials. But the simple method we will be using here, a softmax regression, won't.

![title](https://www.tensorflow.org/versions/master/images/mnist-train-xs.png)
: mnist.train.xs
: mnist.train.images is a tensor (an n-dimensional array) with a shape of [55000, 784]

![title](https://www.tensorflow.org/versions/master/images/mnist-train-ys.png)
: mnist.train.labels
: [55000, 10] - one hot vector

# Softmax Regression

This is a classic case where a softmax regression is a natural, simple model. If you want to assign probabilities to an object being one of several different things, softmax is the thing to do. Even later on, when we train more sophisticated models, the final step will be a layer of softmax.

A softmax regression has two steps: 
 first we add up the evidence of our input being in certain classes, 
 and then we convert that evidence into probabilities.

To tally up the evidence that a given image is in a particular class, we do a weighted sum of the pixel intensities. The weight is negative if that pixel having a high intensity is evidence against the image being in that class, and positive if it is evidence in favor.

The following diagram shows the weights one model learned for each of these classes. Red represents negative weights, while blue represents positive weights.

![title](https://www.tensorflow.org/versions/master/images/softmax-weights.png)

We also add some extra evidence called a bias. Basically, we want to be able to say that some things are more likely independent of the input. The result is that the evidence for a class i given an input x is:

* evidencei=∑jWi, jxj+bi

where Wi is the weights and bi is the bias for class i, and j is an index for summing over the pixels in our input image x. We then convert the evidence tallies into our predicted probabilities y using the "softmax" function:

* y=softmax(evidence)

Here softmax is serving as an "activation" or "link" function, shaping the output of our linear function into the form we want -- in this case, a probability distribution over 10 cases. You can think of it as converting tallies of evidence into probabilities of our input being in each class. It's defined as:

* softmax(x)=normalize(exp(x))

If you expand that equation out, you get:

* softmax(x)i=exp(xi)/∑jexp(xj)

But it's often more helpful to think of softmax the first way: exponentiating its inputs and then normalizing them. 

http://neuralnetworksanddeeplearning.com/chap3.html#softmax

![title](https://www.tensorflow.org/versions/master/images/softmax-regression-scalargraph.png)

![title](https://www.tensorflow.org/versions/master/images/softmax-regression-vectorequation.png)

* y=softmax(Wx+b)

# Implementing the Regression

To do efficient numerical computing in Python, we typically use libraries like NumPy that do expensive operations such as matrix multiplication outside Python, using highly efficient code implemented in another language. Unfortunately, there can still be a lot of overhead from switching back to Python every operation. This overhead is especially bad if you want to run computations on GPUs or in a distributed manner, where there can be a high cost to transferring data.
=> numpy 같은 package가 있지만 data 이동에 따른 overhead가 커서 GPU, 분산환경 등에서 사용하기 적절치 않다.

TensorFlow also does its heavy lifting outside python, but it takes things a step further to avoid this overhead. Instead of running a single expensive operation independently from Python, TensorFlow lets us describe a graph of interacting operations that run entirely outside Python. (Approaches like this can be seen in a few machine learning libraries.)
=> 전체 작업 flow를 외부에 설정하고 처리해야할 data를 넘겨주고 결고만 받아옴.

In [45]:
import tensorflow as tf

In [46]:
# manipulating symbolic variables.
x = tf.placeholder(tf.float32, [None, 784])

a value that we'll input when we ask TensorFlow to run a computation.
We represent this as a 2-D tensor of floating-point numbers, with a shape [None, 784]. (Here None means that a dimension can be of any length.)

In [47]:
## A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations.
## It can be used and even modified by the computation. 
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Notice that W has a shape of [784, 10] because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. b has a shape of [10] so we can add it to the output.

In [48]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

First, we multiply x by W with the expression tf.matmul(x, W). This is flipped from when we multiplied them in our equation, where we had Wx, as a small trick to deal with x being a 2D tensor with multiple inputs. (??)

# Training

In order to train our model, we need to define what it means for the model to be good. Well, actually, in machine learning we typically define what it means for a model to be bad, called the cost or loss, and then try to minimize how bad it is. But the two are equivalent.

One very common, very nice cost function is "cross-entropy."
Hy′(y)=−∑iy′ilog(yi)

Where y is our predicted probability distribution, and y′ is the true distribution (the one-hot vector we'll input).
In some rough sense, the cross-entropy is measuring how inefficient our predictions are for describing the truth.

http://colah.github.io/posts/2015-09-Visual-Information/

In [49]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [50]:
# Then we can implement the cross-entropy, −∑y′log(y):
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

TensorFlow knows the entire graph of your computations, it can automatically use the backpropagation algorithm to efficiently determine how your variables affect the cost you ask it minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the cost.

http://colah.github.io/posts/2015-08-Backprop/

In [51]:
# In this case, we ask TensorFlow to minimize cross_entropy using the gradient descent algorithm with a learning rate of 0.01.
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

TensorFlow also provides many other optimization algorithms: using one is as simple as tweaking one line.

https://www.tensorflow.org/versions/master/api_docs/python/train.html#optimizers

Now we have our model set up to train. One last thing before we launch it, we have to add an operation to initialize the variables we created:

In [52]:
init = tf.initialize_all_variables()

In [53]:
# We can now launch the model in a Session, and run the operation that initializes the variables:
sess = tf.Session()
sess.run(init)

Each step of the loop, we get a "batch" of one hundred random data points from our training set. We run train_step feeding in the batches data to replace the placeholders.

In [54]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Using small batches of random data is called stochastic training -- in this case, stochastic gradient descent.

Ideally, we'd like to use all our data for every step of training because that would give us a better sense of what we should be doing, but that's expensive. So, instead, we use a different subset every time. Doing this is cheap and has much of the same benefit.

# Evaluating Our Model

tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis.

In [55]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [56]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [57]:
print sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9163


Is that good? Well, not really. In fact, it's pretty bad.